TensorFlow 支持 CPU 和 GPU。它也支持分布式计算。可以在一个或多个计算机系统的多个设备上使用 TensorFlow。

TensorFlow 将支持的 CPU 设备命名为“/device：CPU：0”（或“/cpu：0”），第 i 个 GPU 设备命名为“/device：GPU：I”（或“/gpu：I”）。

如前所述，GPU 比 CPU 要快得多，因为它们有许多小的内核。然而，在所有类型的计算中都使用 GPU 也并不一定都有速度上的优势。有时，比起使用 GPU 并行计算在速度上的优势收益，使用 GPU 的其他代价相对更为昂贵。

为了解决这个问题，TensorFlow 可以选择将计算放在一个特定的设备上。默认情况下，如果 CPU 和 GPU 都存在，TensorFlow 会优先考虑 GPU。

TensorFlow 将设备表示为字符串。本节展示如何在 TensorFlow 中指定某一设备用于矩阵乘法的计算。

# 具体做法

In [1]:
import tensorflow as tf

1、要验证 TensorFlow 是否确实在使用指定的设备（CPU 或 GPU）可以创建会话，并将 log_device_placement 标志设置为 True，即：

In [2]:
config = tf.ConfigProto(log_device_placement=True)

2、如果你不确定设备，并希望 TensorFlow 选择现有和受支持的设备，则可以将 allow_soft_placement 标志设置为 True：

In [3]:
config = tf.ConfigProto(allow_soft_placement=True,log_device_placement=True)

3、手动选择 CPU 进行操作：

In [4]:
with tf.device('/cpu:0'):
    rand_t = tf.random_uniform([50,50],0,10,dtype=tf.float32,seed=12)
    a = tf.Variable(rand_t)
    b = tf.Variable(rand_t)
    c = tf.matmul(a,b)
    init = tf.global_variables_initializer()

sess = sess = tf.Session(config=config)
sess.run(init)
print(sess.run(c))
sess.close()

Instructions for updating:
Colocations handled automatically by placer.
[[ 1449.85388184  1626.1751709   1421.05212402 ...,  1325.81970215
   1649.21606445  1351.7434082 ]
 [ 1181.17590332  1334.8984375   1242.13842773 ...,  1164.77185059
   1372.31433105  1023.44702148]
 [ 1379.08691406  1442.31420898  1418.13635254 ...,  1432.7623291
   1523.3260498   1301.82641602]
 ..., 
 [ 1373.33654785  1478.98583984  1334.28039551 ...,  1285.56225586
   1374.23120117  1125.81274414]
 [ 1332.47875977  1423.56616211  1507.86206055 ...,  1280.78063965
   1520.20788574  1213.62231445]
 [ 1400.01989746  1492.82287598  1466.52075195 ...,  1344.32324219
   1631.2947998   1210.82836914]]


手动选择一个 GPU 来操作：

In [5]:
with tf.device('/gpu:0'):
    rand_t = tf.random_uniform([50,50],0,10,dtype=tf.float32,seed=0)
    a = tf.Variable(rand_t)
    b = tf.Variable(rand_t)
    c = tf.matmul(a,b)
    init = tf.global_variables_initializer()

sess = sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
sess.run(init)
print(sess.run(c))
sess.close()

[[ 1405.24291992  1441.74133301  1364.38000488 ...,  1480.22509766
   1279.00610352  1620.09375   ]
 [ 1232.65893555  1344.44580078  1169.70947266 ...,  1205.12841797
   1040.55664062  1421.96704102]
 [ 1209.31640625  1180.32055664  1158.13964844 ...,  1200.03442383
   1014.03216553  1222.51074219]
 ..., 
 [ 1298.96484375  1262.92358398  1205.69177246 ...,  1396.47900391
   1090.7253418   1437.2409668 ]
 [ 1118.24731445  1209.01513672  1077.72290039 ...,  1180.70251465
   1076.46936035  1139.74194336]
 [ 1200.88659668  1297.2265625   1260.01000977 ...,  1289.4296875
   1165.24475098  1433.41833496]]


手动选择多个GPU：

In [9]:
c=[]
for d in ['/gpu:1','/gpu:2']:
    with tf.device(d):
        rand_t = tf.random_uniform([50,50],0,10,dtype=tf.float32,seed=0)
        a = tf.Variable(rand_t)
        b = tf.Variable(rand_t)
        c.append(tf.matmul(a,b))
        init = tf.global_variables_initializer()

sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,log_device_placement=True))
sess.run(init)
print(sess.run(c))
sess.close()

[array([[ 1405.24291992,  1441.74133301,  1364.38000488, ...,
         1480.22509766,  1279.00610352,  1620.09375   ],
       [ 1232.65893555,  1344.44580078,  1169.70947266, ...,
         1205.12841797,  1040.55664062,  1421.96704102],
       [ 1209.31640625,  1180.32055664,  1158.13964844, ...,
         1200.03442383,  1014.03216553,  1222.51074219],
       ..., 
       [ 1298.96484375,  1262.92358398,  1205.69177246, ...,
         1396.47900391,  1090.7253418 ,  1437.2409668 ],
       [ 1118.24731445,  1209.01513672,  1077.72290039, ...,
         1180.70251465,  1076.46936035,  1139.74194336],
       [ 1200.88659668,  1297.2265625 ,  1260.01000977, ...,
         1289.4296875 ,  1165.24475098,  1433.41833496]], dtype=float32), array([[ 1405.24291992,  1441.74133301,  1364.38000488, ...,
         1480.22509766,  1279.00610352,  1620.09375   ],
       [ 1232.65893555,  1344.44580078,  1169.70947266, ...,
         1205.12841797,  1040.55664062,  1421.96704102],
       [ 1209.31640625,  

在这种情况下，如果系统有 3 个 GPU 设备，那么第一组乘法将由'/：gpu：1'执行，第二组乘以'/gpu：2'执行。

# 解读分析

函数 tf.device() 选择设备（CPU 或 GPU）。with 块确保设备被选择并用于其操作。with 块中定义的所有变量、常量和操作将使用在 tf.device() 中选择的设备。

会话配置使用 tf.ConfigProto 进行控制。通过设置 allow_soft_placement 和 log_device_placement 标志，告诉 TensorFlow 在指定的设备不可用时自动选择可用的设备，并在执行会话时给出日志消息作为描述设备分配的输出。